In [ ]:
# --- 
# aims: compare different interpolation methods 
# calls: none 
# modefication history: gmalik, June, 2021; 

# -------- import libraries -------
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl 
import time 
from scipy import interpolate as interp
from pylab import *
from kde_plus_gmm import kde_plus_gmm 
# --------------------------------

# ------ Read data from file ------
path = '/gpfs/fs0/scratch/j/jphickey/jphickey/Boundary_Layer_PNAS_2017/'
for time in range (13,14):

    time_stamp = '%02d' %time
    fname = path + 'restart_010' + time_stamp + '_ydelta_adrian_scalar_omega_uvw_08240_10565.dat'
    testing_file = '/gpfs/fs0/scratch/j/jphickey/g2malik/working_code/grid_interpolation/testf.dat' # temporary file for debugging
    t = open(testing_file, "w")
    f = open (fname, mode = 'r') 
    print('--- the %d th snapshot in time ---'%time)

    # ----- choose the starting position/parameters -----
    btfti = 0.04 # see Wu, Wallace and Hickey 2019, PoF 
    binbin = 50 

    first_z = 1     # start from the position at Z-label = ? {1...513}
    last_z = 513
    first_x = 0     # start from the position at X-label = ? {0...2326}
    last_x = 2326
    first_y = 105   # [1,400] skip the buffer layer -- 100 wall units 
    last_y = 400    
    
    y_interval = last_y - first_y + 1

    wall_units = 2000 
    x_interval = int(11/50 * wall_units)
    total_x_planes = int((last_x-first_x)/x_interval)
    # --------------------------------

    # ------ Calculate x_cordinates -----
    #for aa in range(50):
    #   z_plane = aa * 10 + first_z  # get a slide of data every 10 points in z
    #   for x_plane in range(total_x_planes):
    z_plane = 41
    x_plane = 4
 
    stax = first_x + x_interval * x_plane
    skpx = last_x - x_interval 
    endx = stax + x_interval 
    # --------------------------------

    # ----skip / go to certain index -----
    for i in range(3):
        data = f.readline()
    for ii in range(stax-1):
        data = f.readline()
    for jj in range(first_y-1):
        for ii in range(last_x):
            data = f.readline()
    for kk in range(z_plane-1):
        for jj in range(last_y):
            for ii in range(last_x):
                data = f.readline()
    # --------------------------------
    
    # ------- get velocity etc -------
    xy    = [[],[]] # computational grid
    uu    = []      # computational grid
    tt    = []
    small = 1e-15

    for j in range(y_interval):
        ylb = first_y + j + 1 # y-label grid 
        for i in range(x_interval):
            data = f.readline()
            lst = data.split()
            x = float(lst[0]) - 10842.4  # minus the starting position at re_theta = 1800 
            y = float(lst[1])
            yod = float(lst[3])
            xod = x / (y+small) * yod
            tl = float(lst[5]) # passive scalar, index of btfti 
            u = float(lst[7])
            xy[0].append(xod)
            xy[1].append(yod)
            uu.append(u)
            tt.append(tl)
        for i in range(skpx): # skip the x that we don't need 
            data = f.readline()
    # --------------------------------

    # ----- Raw Contour snapshot of flow -----
    x_raw = np.reshape(xy[0],(-1,x_interval)
    y_raw = np.reshape(xy[1],(-1,x_interval)
    u_raw = np.reshape(uu,(-1,x_interval))
    np.savetxt(testing_file,y_grid)
    
    figure(figsize=(20.0, 3.0), dpi=1000)
    plt.contourf(x_raw[:145][:],y_raw[:145][:],u_raw[:145][:],40,cmap='hot')
    plt.title("010%s raw stream snapshot X#%d at Zlabel = %d "%(time_stamp,x_plane+1,z_plane),fontdict={'family' : 'Calibri', 'size':12})
    plt.savefig('/gpfs/fs0/scratch/j/jphickey/g2malik/working_code/grid_interpolation/Results/gridsize_convergence/raw data stream snapshot.png')
    plt.show()
    plt.close()
    #---------------------------------------
    
    # ----- Creating Uniform Grid -----   
    xmax = xy[0][-1]
    xmin = xy[0][0]
    ymax = xy[1][-1]
    ymin = xy[1][0]

    interpx = 600  # number of pts 
    interpy = 600
    xi=np.linspace(xmin,xmax,interpx)
    yi=np.linspace(ymin,ymax,interpy)  

    XY  = np.meshgrid(xi,yi)
    #---------------------------------------

    # ----- griddata interpolation method ------
    UU  = interp.griddata((xy[0],xy[1]), uu, (XY[0],XY[1]), method =  'cubic') #Uniform interpolated grid where xy is a flat array
    TT  = interp.griddata((xy[0],xy[1]), tt, (XY[0],XY[1]), method =  'cubic') #Uniform interpolated grid

    figure(figsize=(20.0, 3.0), dpi=1000)
    plt.contourf(XY[0][:120][:],XY[1][:120][:],UU[:120][:],40,cmap='hot')
    plt.title("cubic griddata method 010%s X#%d at Zlabel = %d grid points = %d"%(time_stamp,jy,staz,interpx),fontdict={'family' : 'Calibri', 'size':12})
    plt.savefig('/gpfs/fs0/scratch/j/jphickey/g2malik/working_code/grid_interpolation/Results/gridsize_convergence/cubic griddata 010%s snapshot grid_points = %d.png'%(time_stamp,interpx))
    plt.show()
    plt.close()
    # ---------------------------------------

    # ----- rbf interpolation method --------
    rbf_method = interp.rbf(xy[0], xy[1], uu, function='quintic')
    U_rbf = rbf_method(XY[0],XY[1])

    figure(figsize=(20.0, 3.0), dpi=1000)
    plt.contourf(XY[0][:120][:],XY[1][:120][:],U_rbf[:120][:],40,cmap='hot')
    plt.title("quintic rbf method 010%s X#%d at Zlabel = %d grid points = %d"%(time_stamp,jy,staz,interpx),fontdict={'family' : 'Calibri', 'size':12})
    plt.savefig('/gpfs/fs0/scratch/j/jphickey/g2malik/working_code/grid_interpolation/Results/gridsize_convergence/quintic rbf 010%s snapshot grid_points = %d.png'%(time_stamp,interpx))
    plt.show()
    plt.close()
    # ---------------------------------------

    # ----- interp2d interpolation method --------
    interp2d_method = interp.interp2d(xy[0], xy[1], uu, kind='cubic')
    U_interp2d = interp2d_method(xi,yi)

    figure(figsize=(20.0, 3.0), dpi=1000)
    plt.contourf(XY[0][:120][:],XY[1][:120][:],U_interp2d[:120][:],40,cmap='hot')
    plt.title("cubic interp2d method 010%s X#%d at Zlabel = %d grid points = %d"%(time_stamp,jy,staz,interpx),fontdict={'family' : 'Calibri', 'size':12})
    plt.savefig('/gpfs/fs0/scratch/j/jphickey/g2malik/working_code/grid_interpolation/Results/gridsize_convergence/cubic interp2d 010%s snapshot grid_points = %d.png'%(time_stamp,interpx))
    plt.show()
    plt.close()
    # ---------------------------------------

    

t.close()
print ('--- End of all ---')
